<a href="https://colab.research.google.com/github/JotaBlanco/TheValley/blob/main/Arboles/Clase_02_Arboles/02_A_%C3%81rboles_Clasificaci%C3%B3n_M%C3%BAltiple_Resuelto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01 INTRO: Árboles de Decisión
Explicación de cómo construír árboles de decisión de **clasificación multiclase**.

Notebook por [Javier Blanco Cordero](https://www.linkedin.com/in/javier-blanco-cordero-71373656/).

### Enlaces de interés
*   [Slides de presentación](https://docs.google.com/presentation/d/1kiEbdMHy7Ji02SlTxzq913bZ-rcQWn00Td0K_MNVXEk/edit?usp=sharing)
*   [Enlace a este notebook en Colab](https://colab.research.google.com/github/JotaBlanco/TheValley/blob/main/Arboles/Clase_01_Arboles/01_%C3%81rboles_Decisi%C3%B3n_Clasificaci%C3%B3n_Resuelto.ipynb)




## 0101 Qué es un árbol de decisión?
Un tipo de algoritmo de aprendizaje supervisado que se basa en realizar particiones a partir de distintos niveles de las variables disponibles.

## 0102 Import
Importamos todas las librerías necesarias para este análisis ([¿No sabes lo que es una librería de Python?](https://www.quora.com/What-is-a-Python-library-and-what-can-I-use-it-for)): pandas, numpy, seaborn, matplotlib.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## 0103 Carga el dataset: salud del feto
Para probar a hacer árboles de decisión de clasificación multiclas utilizaremos un dataset sobre el estado de salud de un feto que he encontrado en Kaggle ([aquí](https://www.kaggle.com/andrewmvd/fetal-health-classification)). 

Podéis encontrar el archivo listo para importar en mi github: 'https://raw.githubusercontent.com/JotaBlanco/TheValley/main/Data/fetal_health.csv'. 

Importa este dataset en un dataframe llamado **df**.

In [ ]:
# Url archivo raw
url = 'https://raw.githubusercontent.com/JotaBlanco/TheValley/main/Data/fetal_health.csv'

# Importa csv
df = pd.read_csv(url)

# Visualización primeras filas
df.head()

# 02 EDA
Realizaremos un pequeño análisis exploratorio visual para familiarizarnos con el dataset. 

Recuerda que puedes encontrar mis clases sobre análisis exploratorio [aquí](https://github.com/JotaBlanco/TheValley/tree/main/EDA/).

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
# Visualización coeficientes Pearson
plt.figure(figsize=(12,9))
sns.heatmap(np.round(df.corr(),2), 
            vmin=-1, vmax=1, 
            annot=True, cmap="coolwarm")
plt.show()

# 03 Construcción árbol de decisión a mano
Nunca lo haremos en la vida real, pero para afianzar el entendimiento de este tipo de algoritmos, vamos a reproducir el proceso de entrenamiento a mano.

## 0301 DataFrame
Para simplificar, haremos esto con un dataframe consistente solo en las columnas ['baseline value', 'accelerations', 'fetal_movement', 'fetal_health'] renombradas como ['Base', 'Acc', 'Mov', 'Salud Feto'].

In [ ]:
df.columns

In [ ]:
df_resumen = df[['baseline value', 'accelerations', 'fetal_movement', 'fetal_health']]
df_resumen.columns = ['Base', 'Acc', 'Mov', 'Salud Feto']
df_resumen.head(7)

## 0301 Seleccionamos el nodo raíz
Para seleccionar el nodo raíz generaremos árboles muy sencillos, de solo 3 nodos (una sola partición).

### 030101 Variable categórica
En este dataset resumen no tenemos una variable categórica binaria, pero generamos una a partir de una partición sobre una variable continua (por ejemplo Base > 132).

A partir de esa nueva variable categórica binaria probamos a generar una partición.

In [ ]:
df_resumen_i = df_resumen.copy(deep=True)
filtro_i = df_resumen_i['Base'] > 132
df_resumen_i['base > 132'] = filtro_i.astype(int)
df_resumen_i.head()

In [ ]:
# Podemos hacer la partición Anemia = 1 y Anemia = 0 con un groupby
df_i = df_resumen_i.groupby(['base > 132','Salud Feto'])['Base'].agg(['count']).reset_index()
df_i

In [ ]:
def particion_categorica(df, target, var_discreta):
  """
  Genera un dataframe con información sobre la partición utilizando la variable
  discreta binaria aportada.
  """
  df_i = df.groupby([var_discreta,target])[df.columns[0]].agg(['count']).reset_index()
  
  return df_i.sort_values(var_discreta, ascending=True).reset_index(drop=True)

### 030102 Variable Continua
Las variables continuas pueden dar lugar a diversas particiones. Veamos la variable 'Base' como ejemplo.

In [ ]:
def particiones_continuas(df, target, var_continua):
  """
  Muestra una serie dataframes con información sobre las distintas particiones 
  posibles utilizando la variable continua aportada.
  """
  valores_unicos_ordenados = df[var_continua].sort_values().unique()
  valores_particiones = (valores_unicos_ordenados[:-1] + 
                         valores_unicos_ordenados[1:])/2

  for valor in valores_particiones:
    df_i = df.copy(deep=True)
    filtro_i = df_i[var_continua] > valor
    df_i[str(var_continua) + ' > ' + str(valor)] = filtro_i.astype(int)
    df_i = particion_categorica(df_i, 
                                target, 
                                str(var_continua) + ' > ' + str(valor))
    display(df_i)

In [ ]:
particiones_continuas(df_resumen, 'Salud Feto', 'Base')

### 030103 Impureza de Gini
Calculamos la impureza de Gini de cada una de las particiones.

In [ ]:
df_i

In [ ]:
def gini(df_i):
  """
  Calcula el Gini para cada nodo el ponderado de la decisión.
  """
  for nodo in [0, 1]:
    gini = 1
    filtro_condicion = df_i[df_i.columns[0]]==nodo
    for clase in df_i.loc[filtro_condicion, df_i.columns[1]]:
      conteo_i = int(df_i.loc[((filtro_condicion)&(df_i[df_i.columns[1]]==clase)), 'count'])
      prob_i = conteo_i/df_i.loc[filtro_condicion, 'count'].sum()
      gini = gini - prob_i**2
    
    df_i.loc[filtro_condicion,'Gini'] = gini
  
  df_i_group = df_i.groupby([df_i.columns[0], 'Gini'])['count'].agg('sum').reset_index()
  gini_pond = (df_i_group['count'][0] * df_i_group['Gini'][0] +  
               df_i_group['count'][1] * df_i_group['Gini'][1])/(df_i_group['count'].sum())
  
  df_i['Gini Media Pond'] = gini_pond
  return df_i

In [ ]:
gini(df_i)

In [ ]:
def particion_categorica(df, target, var_discreta):
  """
  Genera un dataframe con información sobre la partición utilizando la variable
  discreta binaria aportada.
  """
  df_i = df.groupby([var_discreta,target])[df_resumen_i.columns[0]].agg(['count']).reset_index()
  df_i = df_i.sort_values(var_discreta, ascending=True).reset_index(drop=True)
  return gini(df_i)

### 030104 Cuál es la mejor partición?

In [ ]:
def mejor_particion_continua(df, target, var_continua):
  """
  Devuelve un dataframe con información sobre la partición ópitma de entre las 
  posibles utilizando la variable continua aportada.
  """
  valores_unicos_ordenados = df[var_continua].sort_values().unique()
  valores_particiones = (valores_unicos_ordenados[:-1] + 
                         valores_unicos_ordenados[1:])/2
  particion_optima = []

  for i, valor in enumerate(valores_particiones):
    df_i = df.copy(deep=True)
    filtro_i = df_i[var_continua] > valor
    df_i[str(var_continua) + ' > ' + str(valor)] = filtro_i.astype(int)
    df_i = particion_categorica(df_i, 
                                target, 
                                str(var_continua) + ' > ' + str(valor))

    if i == 0:
      particion_optima = df_i
    else:
      if df_i['Gini Media Pond'][0] < particion_optima['Gini Media Pond'][0]:
        particion_optima = df_i
    
  return particion_optima

In [ ]:
mejor_particion_continua(df_resumen, 'Salud Feto', 'Base')

In [ ]:
mejor_particion_continua(df_resumen, 'Salud Feto', 'Acc')

In [ ]:
mejor_particion_continua(df_resumen, 'Salud Feto', 'Mov')

In [ ]:
def seleccion_particion(df, target, lista_vars_continuas, lista_vars_binarias):
  particion_optima = []

  for i, var in enumerate(lista_vars_continuas+lista_vars_binarias):
    if var in lista_vars_continuas:
      df_i = mejor_particion_continua(df, target, var)
    if var in lista_vars_binarias:
      df_i = particion_categorica(df,  target, var)
    

    if i == 0:
      particion_optima = df_i
    else:
      if df_i['Gini Media Pond'][0] < particion_optima['Gini Media Pond'][0]:
        particion_optima = df_i
    
  return particion_optima

In [ ]:
seleccion_particion(df_resumen, 'Salud Feto', ['Base', 'Acc', 'Mov'], [])

## 0302 Construímos las ramas
Vamos a contruír el resto del árbol sin repetir variables.

In [ ]:
# Realizamos la partición
df_resumen_1 = df_resumen[df_resumen['Acc']>0.0005]
df_resumen_2 = df_resumen[df_resumen['Base']<0.0005]

In [ ]:
df_resumen.columns

### 030201 Rama Izquierda (1)

In [ ]:
# Siguiente partición
seleccion_particion(df_resumen_1, 'Salud Feto', ['Base', 'Acc', 'Mov'], [])

In [ ]:
# Realizamos la siguiente partición
df_resumen_1_1 = df_resumen[((df_resumen['Acc']>0.0005)&(df_resumen['Acc']>0.0025))]
df_resumen_1_2 = df_resumen[((df_resumen['Acc']>0.0005)&(df_resumen['Acc']<0.0025))]

In [ ]:
# Siguiente partición
seleccion_particion(df_resumen_1_1, 'Salud Feto', ['Base', 'Acc', 'Mov'], [])

In [ ]:
# Siguiente partición
seleccion_particion(df_resumen_1_2, 'Salud Feto', ['Base', 'Acc', 'Mov'], [])

# 04 Construcción árbol de decisión con scikit learn
Ahora de manera automática, utilizando la librería [scikit learn](https://scikit-learn.org/stable/), la libería básica de referencia para machine learning.

## 0401 Importamos las librerías

In [ ]:
# Nos importamos las funciones de árboles de scikit learn
from sklearn import tree

## 0402 Inicializamos el árbol
Definiendo las características que tendrá.

https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier

In [ ]:
# Inicializamos un árbol. Por ahora está vacío. 
# Solo definimos cómo queremos que sea en cuanto a su estructura y condiciones de entrenamiento
arbol = tree.DecisionTreeClassifier(max_depth=3)
arbol

## 0403 Entrenamos el modelo
Una vez inicializado y definido el árbol, el método .[fit()](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier.fit) realiza el proceso de particionado recursivo que hemos hecho nosotros antes a mano.

In [ ]:
# Entrenamos el árbol inicializado a partir de los datos que le pasemos con .fit()
arbol = arbol.fit(X = df_resumen[['Base', 'Acc', 'Mov']],  # vars descriptivas
                  y = df_resumen[['Salud Feto']])                  # var objetico
arbol

##0404 Visualizamos el modelo
Una vez entrenado el modelo, podemos visualizarlo.

In [ ]:
# Con el método export_text en un print()
esquema_print = tree.export_text(arbol, 
                                 feature_names= ['Base', 'Acc', 'Mov'])
print(esquema_print)

In [ ]:
# O con la librería graphviz
import graphviz 

# Export_graphviz
dot_data = tree.export_graphviz(arbol, 
                                out_file=None, 
                                feature_names= ['Base', 'Acc', 'Mov']) 
graph = graphviz.Source(dot_data) 
graph

## 0405 Realizamos una predicción
Una vez el modelo está entrenado (definido) podemos pasarle datos para obtener las prediciones correspondientes de la variable objetivo. 

Para esto utilizamos el método .[predict()](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier.predict).

In [ ]:
# Predicción
prediccion = arbol.predict(df_resumen[['Base', 'Acc', 'Mov']])
prediccion

In [ ]:
# Cómo de buena es la predicción?
from sklearn.metrics import accuracy_score
accuracy_score(df_resumen['Salud Feto'], prediccion)

# 05 EJERCICIO
Entrena un árbol de decisión con todas las variables del set de datos de salud del feto, y evalua si la predicción es mejor o peor que utilizando tres variables (que es lo que acabamos de hacer en la sección anterior).

Hazlo primero manteniendo una profundidad de 3 como en el ejemplo anterior y luego prueba también con 5 y 20.

In [ ]:
df.head(2)

In [ ]:
df.columns

In [ ]:
# Inicializamos un árbol. Por ahora está vacío. Solo definimos cómo queremos que sea.
arbol_3 = tree.DecisionTreeClassifier(max_depth=3)
arbol_5 = tree.DecisionTreeClassifier(max_depth=5)
arbol_20 = tree.DecisionTreeClassifier(max_depth=20)

In [ ]:
# Matrices de datos
cols = ['baseline value', 'accelerations', 'fetal_movement',
       'uterine_contractions', 'light_decelerations', 'severe_decelerations',
       'prolongued_decelerations', 'abnormal_short_term_variability',
       'mean_value_of_short_term_variability',
       'percentage_of_time_with_abnormal_long_term_variability',
       'mean_value_of_long_term_variability', 'histogram_width',
       'histogram_min', 'histogram_max', 'histogram_number_of_peaks',
       'histogram_number_of_zeroes', 'histogram_mode', 'histogram_mean',
       'histogram_median', 'histogram_variance', 'histogram_tendency']
X = df[cols]
y = df['fetal_health']

In [ ]:
# Entrenamos los árboles inicializados a partir de los datos que le pasemos con .fit()
arbol_3 = arbol_3.fit(X = X, y = y)
arbol_5 = arbol_5.fit(X = X, y = y)
arbol_20 = arbol_20.fit(X = X, y = y)

In [ ]:
# Podemos visualizar la estructura
print(tree.export_text(arbol_3, 
                       feature_names=list(X.columns)))

In [ ]:
# Podemos visualizar la estructura
print(tree.export_text(arbol_5, 
                       feature_names=list(X.columns)))

In [ ]:
import graphviz 
dot_data = tree.export_graphviz(arbol_5, 
                                out_file=None, 
                                feature_names=list(X.columns)) 
graph = graphviz.Source(dot_data) 
graph

In [ ]:
# Predicción
prediccion_3 = arbol_3.predict(X)
prediccion_5 = arbol_5.predict(X)
prediccion_20 = arbol_20.predict(X)

In [ ]:
# Precisión de la predicción
print('Precisión de arbol depth=3 es de ',accuracy_score(y, prediccion_3))
print('Precisión de arbol depth=5 es de ',accuracy_score(y, prediccion_5))
print('Precisión de arbol depth=20 es de ',accuracy_score(y, prediccion_20))